1. 数据加载
将数据文件加载到 Pandas 数据框中。

In [1]:
import pandas as pd

# 数据文件目录
data_dir = "2025_Problem_C_Data"

# 读取数据
try:
    medal_counts_df = pd.read_csv(f"{data_dir}/summerOly_medal_counts.csv")
    programs_df = pd.read_csv(f"{data_dir}/summerOly_programs.csv")
except FileNotFoundError:
    print(f"错误：找不到数据文件。请确保 '{data_dir}' 目录下包含所需 CSV 文件。")
    exit()  # 如果文件不存在，则退出程序
# Explore the dataframes
print("Medal Counts data:")
print(medal_counts_df.head())  # View the first few rows of the medal_counts dataframe

print("\nProgram data:")
print(programs_df.head())  # View the first few rows of the programs dataframe

# You can now start your data analysis using the dataframes

# For example, to calculate the total number of medals awarded each year, you can group the medal_counts_df by 'Year' and sum the 'Total' column

Medal Counts data:
   Rank            NOC  Gold  Silver  Bronze  Total  Year
0     1  United States    11       7       2     20  1896
1     2         Greece    10      18      19     47  1896
2     3        Germany     6       5       2     13  1896
3     4         France     5       4       2     11  1896
4     5  Great Britain     2       3       2      7  1896

Program data:
      Sport         Discipline Code Sports Governing Body 1896 1900 1904  \
0  Aquatics  Artistic Swimming  SWA        World Aquatics    0    0    0   
1  Aquatics             Diving  DIV        World Aquatics    0    0    2   
2  Aquatics  Marathon Swimming  OWS        World Aquatics    0    0    0   
3  Aquatics           Swimming  SWM        World Aquatics    4    7    9   
4  Aquatics         Water Polo  WPO        World Aquatics    0    1    1   

   1906*  1908 1912  ...  1988 1992  1996  2000  2004  2008  2012  2016  2020  \
0      0     0    0  ...     2    2   1.0   2.0   2.0   2.0   2.0   2.0   2.0   

2. 数据清洗
将项目数据转换为长格式，方便后续分析

将奖牌数据和项目数据合并


In [4]:
# 数据清洗：将项目数据转换为长格式
program_df_melted = programs_df.melt(
    id_vars=["Sport", "Discipline", "Code", "Sports Governing Body"],
    var_name="Year",
    value_name="Events",
)

# 将年份转换为数值类型，并处理非数值年份（例如 '1906*'）
program_df_melted["Year"] = program_df_melted["Year"].str.replace(
    "*", "", regex=False
)  # 移除*号
program_df_melted["Year"] = pd.to_numeric(program_df_melted["Year"], errors="coerce")
program_df_melted = program_df_melted.dropna(subset=["Year"])  # 删除年份为空的行

# 将奖牌数据中的年份也转换为数值类型
medal_counts_df["Year"] = pd.to_numeric(medal_counts_df["Year"])

# 合并奖牌数据和项目数据
merged_df = pd.merge(medal_counts_df, program_df_melted, on="Year", how="left")

3. 分析比赛项目的奖牌分布
分组统计：按项目和国家分组，计算各项目奖牌数量的分布。

In [5]:
# --- 数据分析 ---

# 1. 各国历年奖牌总数
medal_by_noc_year = merged_df.groupby(["NOC", "Year"])["Total"].sum().unstack()
print("\n各国历年奖牌总数：\n", medal_by_noc_year)

# 2. 各国在不同项目的奖牌总数
medal_by_noc_sport = merged_df.groupby(["NOC", "Sport"])["Total"].sum().unstack()
print("\n各国各项目奖牌总数：\n", medal_by_noc_sport)

# 3. 哪些项目被哪些国家统治 (例如：某国在该项目奖牌数占该项目总奖牌数的一定比例以上)
sport_domination = merged_df.groupby(["Sport", "NOC"])["Total"].sum().unstack()
if (
    sport_domination.shape[1] > 0
):  # Check if there are any countries to avoid division by zero
    sport_domination_percentage = (
        sport_domination.div(sport_domination.sum(axis=1), axis=0) * 100
    )
    print("\n各项目各国奖牌占比（百分比）：\n", sport_domination_percentage)
else:
    print("没有数据进行项目统治分析。")

# 4. 识别特色项目 (例如：某项目奖牌数占该国家奖牌总数的一定比例以上)
noc_total_medals = merged_df.groupby("NOC")["Total"].sum()
noc_sport_medals = merged_df.groupby(["NOC", "Sport"])["Total"].sum()
if (
    not noc_total_medals.empty and not noc_sport_medals.empty
):  # Check if the dataframes are empty
    characteristic_sports = noc_sport_medals.div(noc_total_medals, level="NOC") * 100
    print("\n各国特色项目占比（百分比）：\n", characteristic_sports)
else:
    print("没有数据进行特色项目分析。")

# 5. 分析：哪些项目竞争激烈 (例如：多个国家在该项目奖牌数接近)
if (
    sport_domination.shape[1] > 1
):  # Check if there are at least two countries for std calculation
    competitive_sports = sport_domination.apply(lambda x: x.std(), axis=1)
    print("\n各项目竞争激烈程度（标准差）：\n", competitive_sports)
else:
    print("只有一个或零个国家参与，无法计算项目竞争激烈程度。")

# 6. 计算金银铜牌的分布情况
medal_type_distribution = merged_df.groupby("NOC")[["Gold", "Silver", "Bronze"]].sum()
print("\n各国金银铜牌分布：\n", medal_type_distribution)

# 7. 计算每个项目的奖牌总数
sport_total_medals = merged_df.groupby("Sport")["Total"].sum()
print("\n各项目奖牌总数：\n", sport_total_medals)

# 8. 计算每年的奖牌总数
total_medals_per_year = medal_counts_df.groupby("Year")["Total"].sum()
print("\n每年的奖牌总数：\n", total_medals_per_year)


各国历年奖牌总数：
 Year          1896  1900  1904  1908  1912  1920   1924   1928   1932   1936  \
NOC                                                                            
Afghanistan    NaN   NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN   
Albania        NaN   NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN   
Algeria        NaN   NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN   
Argentina      NaN   NaN   NaN   NaN   NaN   NaN  444.0  518.0    NaN  518.0   
Argentina      NaN   NaN   NaN   NaN   NaN   NaN    NaN    NaN  296.0    NaN   
...            ...   ...   ...   ...   ...   ...    ...    ...    ...    ...   
West Germany   NaN   NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN   
Yugoslavia     NaN   NaN   NaN   NaN   NaN   NaN  148.0  370.0    NaN   74.0   
Yugoslavia     NaN   NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN   
Zambia         NaN   NaN   NaN   NaN   NaN   NaN    NaN    NaN    NaN    NaN   
Zimbabwe       NaN   NaN   N

4. 识别特色项目
国家主导项目：识别某国家在特定项目上占据统治地位（如奖牌数占全球此项目奖牌的大部分）。

In [ ]:
# 计算某国家奖牌占某项目的全球比例
project_share = (
    medal_data.groupby(["Sport", "NOC"])["Total"].sum()
    / medal_data.groupby("Sport")["Total"].sum()
)

# 筛选比例高于阈值（如 50%）的记录
dominant_projects = project_share[project_share > 0.5]

项目特色贡献：某项目在某国家奖牌总数中的占比。

In [ ]:
# 筛选单一项目贡献超过阈值（如 30%）的国家
feature_projects = project_medal_distribution[
    project_medal_distribution["Percentage"] > 0.3
]

5. 探究新项目的影响
提取新增项目数据：通过比较历史数据，识别新增的比赛项目。
分析新增项目奖牌变化：统计新增项目对特定国家奖牌总数的影响。

In [ ]:
# 比较新增项目对奖牌数量的贡献
new_project_medals = (
    medal_data[medal_data["Year"] >= 2020].groupby("Sport")["Total"].sum()
)

# 按国家分析新增项目的奖牌分布
new_project_country = medal_data[medal_data["Sport"].isin(new_project_medals.index)]

6. 数据可视化
条形图：展示某国在不同项目中的奖牌贡献。
热力图：展示各项目的全球奖牌分布情况。

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 条形图：展示国家在各项目的奖牌分布
medal_data.groupby("Sport")["Gold"].sum().plot(kind="bar", figsize=(10, 6))
plt.title("Gold Medals by Sport")
plt.show()

# 热力图：展示项目和国家的奖牌分布
sns.heatmap(project_medal_distribution, annot=True, fmt="d", cmap="YlGnBu")
plt.title("Medal Distribution Heatmap")
plt.show()